In [ ]:
import pickle

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
class shower:
    
    def __init__(self,showerId=-1):
        self.N_XMAX=16
        self.N_YMAX=16
        self.X_OFFSET=8
        self.Y_OFFSET=8
        self.showerId=showerId
        self.showerPixels=np.zeros((self.N_XMAX,self.N_YMAX))
        pass
    
    def setShowerEnergy(self,E):
        self.showerEnergy=E
        
    def setPixel(self,i,j,nPhoton):
        if i+self.X_OFFSET >= self.N_XMAX:
            raise Exception(" ",i," > ",self.N_XMAX)
        if j+self.Y_OFFSET >= self.N_YMAX:
            raise Exception(" ",j," > ",self.N_YMAX)
        self.showerPixels[i+self.X_OFFSET][j+self.Y_OFFSET] = nPhoton
        

In [ ]:
class BaseCorsikaDataset:
    
    def __init_( self ):
        pass
    
    def setFileNames(self , fileNames):
        self.fileList=list(fileNames)
    
    def setEnergyFileNames(self,fileNames):
        self.energyFileNames=[i for i in fileNames]
    
    def loadEnergyData(self):
        
        pass
        
    

In [ ]:
txt=[]
enFnames=[]
with open('enFiles.fls') as f:
    txt=f.readlines()
for l in txt:
    enFnames.append(l[:-1])

In [ ]:
showerEnergyMap={}
for fname in enFnames:
    with open(fname) as f:
        print("Processing ",fname)
        txt=f.readlines()
        for l in txt:
            if l[0]=='#':
                continue
            items=l[:-1].split('\t')
            showerEnergyMap[int(items[0])] = float( items[1])
        print("\tLoaded ",len(showerEnergyMap)," shower energies")

In [ ]:
vals=[ i for i in showerEnergyMap.values()]
plt.hist(vals,bins=np.linspace(0,1e3,20))
plt.semilogy()

In [ ]:
showerListPrimary={}
showerListWithNSB={}
nShower=0
NMAX=2000
with open('../workarea/datasets/Gamma/ga_0001_t_corr_bin_nsb.dat') as f:
    l=f.readline()
    currentShower=None
    while l:
        items=l.split('\t')
        showerId      =int(items[0])
        showerI       =int(items[1])
        showerJ       =int(items[2])
        showerNPhotons=int(items[3])
        if not currentShower or (showerId != currentShower.showerId ):
            currentShower=shower(showerId)
            showerListWithNSB[showerId]=currentShower
            nShower+=1
            if nShower%500==0:
                print("processed ",nShower," showers ")
            if nShower > NMAX:
                break
        if showerNPhotons >0:
            l=f.readline()
        currentShower.setPixel(showerI,showerJ,showerNPhotons)
        l=f.readline()
        
print("\nProcesssing the primary showers")        
nShower=0       
with open('../workarea/datasets/Gamma/ga_0001_t_corr_bin.dat') as f:
    l=f.readline()
    currentShower=None
    while l:
        items=l.split('\t')
        showerId      =int(items[0])
        showerI       =int(items[1])
        showerJ       =int(items[2])
        showerNPhotons=int(items[3])
        if not currentShower or (showerId != currentShower.showerId ):
            currentShower=shower(showerId)
            showerListPrimary[showerId]=currentShower
            nShower+=1
            if nShower%500==0:
                print("processed ",nShower," showers ")
            if nShower > NMAX:
                break
        if showerNPhotons >0:
            l=f.readline()
        currentShower.setPixel(showerI,showerJ,showerNPhotons)
        l=f.readline()        

In [ ]:

showerId=np.random.randint(0,len(showerListWithNSB))
# showerId=649
f=plt.figure( figsize=(16,8) )
plt.gcf().set_size_inches(20, 20)        
ax1=plt.subplot(131)
ax2=plt.subplot(132)
ax3=plt.subplot(133)
mat=showerListWithNSB[showerId].showerPixels
ax1.imshow(mat,cmap=plt.cm.Greys)
for i in range(16):
    for j in range(16):
        c = mat[j,i]
        ax1.text(i, j, str(int(c)), va='center', ha='center',color='r')
ax1.set_title(f'Shower ID : {showerId} [ signal + NSB ]',fontsize=15)       

mat=showerListPrimary[showerId].showerPixels
ax2.imshow(mat,cmap=plt.cm.Greys)
for i in range(16):
    for j in range(16):
        if mat[j,i]==0:
            continue
        c = mat[j,i]
        ax2.text(i, j, str(int(c)), va='center', ha='center',color='r')
ax2.set_title(f'Shower ID : {showerId} [ signal ]',fontsize=15)       
diffMat=showerListWithNSB[showerId].showerPixels-showerListPrimary[showerId].showerPixels
ax3.imshow(diffMat,cmap=plt.cm.Greys)
for i in range(16):
    for j in range(16):
        c = diffMat[j,i]
        ax3.text(i, j, str(int(c)), va='center', ha='center',color='r')
ax3.set_title(f'Shower ID : {showerId} [ NSB ]',fontsize=15)       
        

In [ ]:
signal_mat.shape

In [ ]:

plt.gcf().set_size_inches(20, 20)  
ax1=plt.subplot(131)
ax2=plt.subplot(132)
ax3=plt.subplot(133)

# showerId  = 1487
signal_mat=showerListPrimary[showerId].showerPixels
realstic_mat=showerListWithNSB[showerId].showerPixels
axList  = [ ax1, ax2, ax3]
matList = [
            signal_mat ,
            realstic_mat ,
            realstic_mat - signal_mat
          ] 

y0,x0=np.unravel_index(signal_mat.argmax(), signal_mat.shape)
ys, xs =np.where(signal_mat > 0)
y0 = min(ys) ; y1 = max(ys)
x0 = min(xs) ; x1 = max(xs)
w = x1 - x0 + 2 ; h = y1 - y0 + 2
cX = 0.5*( x0 + x1) ; cY = 0.5*( y0 + y1)

L,H = signal_mat.shape
print(1,cX/L,cY/H,w/L,h/L)

for ax,mat in zip(axList,matList):
    ax.imshow(mat,cmap=plt.cm.Greys)
    # plt.grid()

    xs=[cX -0.5*w , cX +0.5*w , cX +0.5*w , cX -0.5*w , cX -0.5*w]
    ys=[cY -0.5*h , cY -0.5*h , cY +0.5*h , cY +0.5*h , cY -0.5*h]
    ax.plot(xs, ys, color="red")
    ax.axis('off')
    ax.set_xticks([])
    ax.set_yticks([])
    
    

## Export as YOLO Dataset Image

In [ ]:
base='../workarea/datasets/yolo/train/'

In [ ]:
signal_mat=showerListPrimary[showerId].showerPixels
realstic_mat=showerListWithNSB[showerId].showerPixels
y0,x0=np.unravel_index(signal_mat.argmax(), signal_mat.shape)
ys, xs =np.where(signal_mat > 0)
y0 = min(ys) ; y1 = max(ys)
x0 = min(xs) ; x1 = max(xs)
w = x1 - x0 + 2 ; h = y1 - y0 + 2
cX = 0.5*( x0 + x1) ; cY = 0.5*( y0 + y1)
L,H = signal_mat.shape
tagStr=f"{showerId} {cX/L} {cY/H} {w/L} {h/L}"
plt.imshow(mat,cmap=plt.cm.Greys)
plt.gca().axis('off')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
foutName=base+f'/{showerId}'
print(f"Saving {foutName}")
with open(foutName+'.txt','w') as f:
    f.write(tagStr)
plt.savefig(foutName+'.png', bbox_inches='tight', pad_inches=0)


In [ ]:
base